# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
 
# Import API key
from api_keys import geoapify_key

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/lauren-ashleyrutland/Desktop/API-Challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,guerrero negro,27.9769,-114.0611,18.40,57,100,8.75,MX,1709754328
1,1,avarua,-21.2078,-159.7750,28.03,78,40,6.17,CK,1709754329
2,2,grytviken,-54.2811,-36.5092,7.08,57,100,1.06,GS,1709754329
3,3,college,64.8569,-147.8028,-6.63,71,75,2.57,US,1709753747
4,4,metlili chaamba,32.2667,3.6333,17.52,32,0,2.57,DZ,1709754330


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [100]:

%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    color = "City",
    legend = True
)

# Configure the map plot

map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [57]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
83,83,trujillo,-8.1160,-79.0300,23.98,83,0,4.12,PE,1709754352
96,96,aswan,24.0934,32.9070,24.61,14,0,1.54,EG,1709754355
117,117,sur,22.5667,59.5289,24.28,75,0,0.74,OM,1709754361
292,292,chui,-33.6971,-53.4616,21.14,48,0,4.30,UY,1709754417
345,345,springs,-26.2500,28.4000,25.93,68,0,3.20,ZA,1709754434
347,347,santiago de cao,-7.9589,-79.2392,24.07,83,0,4.12,PE,1709754434
430,430,tindouf,27.6711,-8.1474,26.25,10,0,1.03,DZ,1709754458
480,480,orkney,-26.9802,26.6727,22.43,20,0,3.04,ZA,1709754474
500,500,roodepoort,-26.1625,27.8725,25.22,56,0,1.30,ZA,1709754482
523,523,salalah,17.0151,54.0924,25.91,72,0,1.43,OM,1709754488


### Step 3: Create a new DataFrame called `hotel_df`.

In [92]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

#Create dictionary variables 

city= city_data_df
country= city_data_df
lat=city_data_df
lng=city_data_df
humidity=city_data_df


#create dictionary 

hotel_dict = {
    'City': city_data_df['City'],
    'Country': city_data_df['Country'],
    'Lat': city_data_df['Lat'],
    'Lng': city_data_df['Lng'],
    'Humidity': city_data_df['Humidity']
}

hotel_df = pd.DataFrame(hotel_dict)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
83,trujillo,PE,-8.1160,-79.0300,83,
96,aswan,EG,24.0934,32.9070,14,
117,sur,OM,22.5667,59.5289,75,
292,chui,UY,-33.6971,-53.4616,48,
345,springs,ZA,-26.2500,28.4000,68,
347,santiago de cao,PE,-7.9589,-79.2392,83,
430,tindouf,DZ,27.6711,-8.1474,10,
480,orkney,ZA,-26.9802,26.6727,20,
500,roodepoort,ZA,-26.1625,27.8725,56,
523,salalah,OM,17.0151,54.0924,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [93]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row ['Lat']
    lon = row ['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
trujillo - nearest hotel: Hostal Ayacucho
aswan - nearest hotel: Oscar Hotel
sur - nearest hotel: Sur Hotel
chui - nearest hotel: Alerces
springs - nearest hotel: Manhattan Hotel & Action Bar
santiago de cao - nearest hotel: Hostal El Encanto
tindouf - nearest hotel: محمد بوسبي
orkney - nearest hotel: No hotel found
roodepoort - nearest hotel: Town Lodge
salalah - nearest hotel: Muscat International Hotel
joshua - nearest hotel: FairBridge Inn & Suites Cleburne


,City,Country,Lat,Lng,Humidity,Hotel Name
83,trujillo,PE,-8.1160,-79.0300,83,Hostal Ayacucho
96,aswan,EG,24.0934,32.9070,14,Oscar Hotel
117,sur,OM,22.5667,59.5289,75,Sur Hotel
292,chui,UY,-33.6971,-53.4616,48,Alerces
345,springs,ZA,-26.2500,28.4000,68,Manhattan Hotel & Action Bar
347,santiago de cao,PE,-7.9589,-79.2392,83,Hostal El Encanto
430,tindouf,DZ,27.6711,-8.1474,10,محمد بوسبي
480,orkney,ZA,-26.9802,26.6727,20,No hotel found
500,roodepoort,ZA,-26.1625,27.8725,56,Town Lodge
523,salalah,OM,17.0151,54.0924,72,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [99]:
%%capture --no-display

# Configure the map plot

map_plot_2 = hotel_df.hvplot.points(
    
# YOUR CODE HERE
    'Lng', 
    'Lat',
    geo= True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',   
    color='City',
    alpha=0.7,
    legend=True,
    hover_cols=['Hotel Name', 'Country']
)


map_plot_2    


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)